### Get all flight codes from OpenFlight website
https://openflights.org/data.php#license

##### Import

In [ ]:
import pandas as pd
import os

In [ ]:
raw_airport_data_path = "raw_airports.csv"

##### Read file

In [ ]:
df = pd.read_csv(raw_airport_data_path, header=None)

##### Rename Column

In [ ]:
rename_columns = {
    0: 'airport_id', 
    1: 'name', 
    2: 'city', 
    3: 'country', 
    4: 'IATA', 
    5: 'ICAO', 
    6: 'latitude', 
    7: 'longitude', 
    8: 'altitude', 
    9: 'timezone', 
    10: 'dst', 
    11: 'tz_database',
    12: 'type', 
    13: 'source'}

In [ ]:
df = df.rename(columns=rename_columns)

##### Replace na values to None

In [ ]:
df_replaced_none = df.replace('\\N', None)

In [ ]:
df_replaced_none

In [ ]:
df_replaced_none.info()

##### Get relevant columns

In [ ]:
df_relevant_columns = df_replaced_none[['name', 'city', 'country', 'IATA', 'ICAO']]

##### Save file to data folder

In [ ]:
airport_code_path = os.path.dirname(os.path.dirname(os.getcwd())) + '\\data\\'

In [ ]:
airport_code_path

In [ ]:
df_relevant_columns.to_csv(airport_code_path + 'all_airport_code.csv', index=False)